# Analytic solution for $T\to 0$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

import os, sys
rootpath = os.path.join(os.getcwd(), '.')
sys.path.append(rootpath)
from src.theory import *
from src.analysis import *
# reimport modules if they change
%load_ext autoreload
%autoreload 2

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Parameters

In [2]:
# specify system parameters
params = {
    'N': 10000, # number of nodes
# not required for mean-field:    'K': 100, # number of connections per node
    'mu': 0.2, # fraction of nodes that receive input
    'sigma': 0.01, # std of additive Gaussian noise
    'epsilon': 0.1, # error threshold for overlap
}
#dt = 1 # fixed time step

# this is resolution on x-axis in results figure (touch with care)
lams = 1 - 10 ** np.linspace(0, -4, 64 + 1)

filename = f"results/results_analytic_0_N{params['N']}_mu{params['mu']}_epsilon{params['epsilon']}_sigma{params['sigma']}.txt"

if os.path.exists(filename):
    raise ValueError("File already exists")

ValueError: File already exists

# Numerically evaluate the solution to the Fokker Planck solution

This is done for integer values of the activity $x$ taking the continuum limit in time $t\to 0$, i.e, looking at the instantaneous number of active neurons in the coupled system

In [3]:
?pmf_from_coupled_fokker_planck

Signature: pmf_from_coupled_fokker_planck(params, h, lam, return_only_pmf=False)
Docstring:
Solution to the Mean-field coupled Fokker-Planck equations. 
1.1) compute solution of FP equation of the part that receives input assuming a mean-field coupling to the recurrently coupled rest
ight)}$$
from mean-field assumption
$$ x_{rest} = rac{(1-\mu)\lambda x_{in}}{1-(1-\mu)\lambda}$$ 

1.2) compute solution of FP equation for the part that does not receive input assuming a mean-field coupling to the input part
ight)}$$
with 
$$ x_{in} = \murac{N p_\mathrm{ext} + \lambda (1-p_\mathrm{ext}) x_\mathrm{rest}}{1-\mu\lambda(1-p_\mathrm{ext})} $$
2) convolution of the two solutions to obtain the full pmf
File:      ~/git/projects/sahel_finite-observation-dynamic-range/src/theory.py
Type:      function

In [31]:
# check the support
x_gauss = support_gauss(bound= 10*params["N"] * params["sigma"], delta=1000)
print(x_gauss)
x = np.arange(0, params["N"] + 1)
x_conv= support_conv_pmf_gauss(x, x_gauss)
print(x_conv)

[-1000.     0.  1000.]
[-1000.     0.  1000.  2000.  3000.  4000.  5000.  6000.  7000.  8000.
  9000. 10000. 11000.]


In [38]:
# gaussian noise (cutoff determines the dynamic range, 5 sigma at 4th digit after comma in test cases)
gauss_std = int(5 * params["N"] * params["sigma"] )
x_gauss = support_gauss(bound=int(5 * params["N"] * params["sigma"] ), delta=1)
pmf_gauss = stats.norm.pdf(x_gauss, 0, params["N"] * params["sigma"])

# final support (pmf from coupled fokker planck has always support [0:N])
x = np.arange(0, params["N"] + 1)
x_conv = support_conv_pmf_gauss(x, x_gauss)
# x_conv = np.arange(np.min(x) + support_gauss[0], np.max(x) + support_gauss[-1] + 1)
indices_x = np.searchsorted(x_conv, x)
pmf = np.zeros_like(x_conv, dtype=np.float64)

def pmf_noise(lam, h):
    pmf[indices_x] = pmf_from_coupled_fokker_planck(params, h=h, lam=lam, return_only_pmf=True)
    return np.convolve(pmf, pmf_gauss, mode="same")

def analysis(lam, verbose=False):
    start = time.time()
    pmf_o_given_h = lambda h: pmf_noise(lam, h)

    # get h_range based on useful bounds from mean-field solution
    h_range = h_range_0(lam, params, verbose=verbose)
    if h_range[0] >= h_range[1]:
        return lam, np.nan, np.nan

    # get refernce distributions from mean-field solution
    pmf_refs = [stats.norm.pdf(x_conv, params["N"] * mean_field_activity(lam, params["mu"], h), params["N"] * params["sigma"]) for h in [0, np.inf]]

    # get dynamic range and number of discriminable states
    dr, nd = analysis_dr_nd(pmf_o_given_h, h_range, pmf_refs, params["epsilon"], verbose=verbose)
    
    end = time.time()
    if verbose:
        print(f"lam={lam:.2f} took {end - start:.2f}s")
    return lam, dr, nd

In [39]:
#check for lam=0 (time = 5min)
analysis(lam=0, verbose=True)

lambda: 0, h_range: (0.0010001000133352234, 6.907755278982136)
possible solution: h=0.13825024402974442 with overlap to end of 3.2900271621207044e-28 ... accepted
possible solution: h=0.300035899298774 with overlap to end of 5.4325463185394236e-18 ... accepted
possible solution: h=0.4944420210377354 with overlap to end of 4.4311555902046926e-11 ... accepted
possible solution: h=0.736746528084268 with overlap to end of 9.917572054506426e-07 ... accepted
possible solution: h=1.0569984962303443 with overlap to end of 0.00029173554101660754 ... accepted
possible solution: h=1.528656749534839 with overlap to end of 0.015670180237130137 ... accepted
possible solution: h=2.4408580031269254 with overlap to end of 0.19216974230407435 ... rejected
possible solution: h=2.0509193526293084 with overlap to end of 3.2241890466514863e-28 ... accepted
possible solution: h=1.3520783662201077 with overlap to end of 5.396403863305802e-18 ... accepted
possible solution: h=0.942685244438958 with overlap to 

(0, 11.712826765754418, 6.0)

In [ ]:
(0, 11.712826765754418, 6.0)



In [36]:
# check for lam=0.99
analysis(lam=0.99, verbose=True)

lambda: 0.99, h_range: (1.0001990395668491e-05, 2.3969857684155342)
possible solution: h=0.0017584018302334301 with overlap to end of 8.25625784110813e-87 ... accepted
possible solution: h=0.0047670924589665155 with overlap to end of 6.207167107641741e-78 ... accepted
possible solution: h=0.00921116737086565 with overlap to end of 6.492407157251877e-69 ... accepted
possible solution: h=0.015331939481662499 with overlap to end of 5.080223373304993e-60 ... accepted
possible solution: h=0.023475884148359577 with overlap to end of 1.8392474295757784e-51 ... accepted
possible solution: h=0.03412766953281576 with overlap to end of 2.201102237268331e-43 ... accepted
possible solution: h=0.04796477721035327 with overlap to end of 6.854752701533578e-36 ... accepted
possible solution: h=0.06594845690186245 with overlap to end of 4.679915060349009e-29 ... accepted
possible solution: h=0.08948191452564223 with overlap to end of 6.17756294212511e-23 ... accepted
possible solution: h=0.1207026950795

(0.99, 27.378370717802333, 15.0)

In [37]:
# check for lam=0.9999
analysis(lam=0.9999, verbose=True)

lambda: 0.9999, h_range: (1.0002000272577614e-07, 0.09521926658097167)
possible solution: h=0.000152091098697984 with overlap to end of 0.0002378222918400284 ... accepted
possible solution: h=0.0017872234626689141 with overlap to end of 0.015222416196002325 ... accepted
possible solution: h=0.01719347108299439 with overlap to end of 0.1891870942689294 ... rejected
possible solution: h=0.008176087689212706 with overlap to end of 8.07692438841779e-27 ... accepted
possible solution: h=0.0008452834837673025 with overlap to end of 7.911263572710966e-05 ... accepted
possible solution: h=8.096489448294658e-05 with overlap to end of 0.23527294893451933 ... rejected
lam=1.00 took 1.68s


(0.9999, 17.304417434341897, 2.0)

In [ ]:
# 65 points took 50min with full symmetric support, now takes 2.5min almost identical result
from dask.distributed import Client, LocalCluster, as_completed
# execute independent lambda computations in parallel with dask
cluster = LocalCluster()
dask_client = Client(cluster)

futures = dask_client.map(analysis, lams)

# run analysis
data = []
for future in tqdm(as_completed(futures), total=len(lams)):
    data.append(future.result())

# sort data by first column
data = np.array(sorted(data, key=lambda x: x[0]))

/Users/johannes/local_build/miniconda3/envs/finite-observation/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63975 instead
  warnings.warn(
100%|██████████| 65/65 [02:41<00:00,  2.48s/it]


In [ ]:
data

array([[ 0.        , 11.71282677,  6.        ],
       [ 0.13403568, 12.52555732,  7.        ],
       [ 0.25010579, 13.315182  ,  8.        ],
       [ 0.35061837, 14.07835876,  9.        ],
       [ 0.43765867, 14.81664097, 10.        ],
       [ 0.51303247, 15.5313915 , 11.        ],
       [ 0.5783035 , 16.22380629, 12.        ],
       [ 0.63482587, 16.89498878, 13.        ],
       [ 0.68377223, 17.54590836, 14.        ],
       [ 0.72615804, 18.17732337, 15.        ],
       [ 0.76286263, 18.78993933, 16.        ],
       [ 0.7946475 , 19.38425167, 17.        ],
       [ 0.82217206, 19.96071268, 17.        ],
       [ 0.84600735, 20.51955947, 18.        ],
       [ 0.86664786, 21.06099594, 18.        ],
       [ 0.8845218 , 21.58493086, 19.        ],
       [ 0.9       , 22.09126423, 19.        ],
       [ 0.91340357, 22.57988305, 20.        ],
       [ 0.92501058, 23.05034971, 20.        ],
       [ 0.93506184, 23.50226803, 20.        ],
       [ 0.94376587, 23.93512033, 20.   

In [ ]:
os.makedirs(os.path.dirname(filename), exist_ok=True)
# save data to file
np.savetxt(
    filename,
    data,
    delimiter="\t",
    header="#lambda\tnumber of discriminable inputs\tdynamic_range",
    comments="",
)